In [1]:
from PIL import Image
from pathlib import Path
import pandas as pd
from matplotlib import pyplot as plt

from utils import get_class, plot_to_tensorboard, evaluate, param_counter, CNNClassifier
from CustomImageDataset import CustomImageDataset
from MLPClassifier import MLPClassifier

import os
import io
import numpy as np
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

from tqdm.notebook import tqdm

import mlflow
import mlflow.pytorch

import warnings

/home/herstegen/ITBA/redes/RN-TP1/.conda/lib/python3.12/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.7'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
logging.getLogger("mlflow").setLevel(logging.ERROR)

## Load dataset

> Cargar imagenes al dataframe

In [3]:
data_dir = r'data/Split_smol/train/'
p = Path(data_dir).glob('**/*')
files = [(x, get_class(x), Image.open(x).size,Image.open(x)) for x in p if x.is_file()]
df_train = pd.DataFrame(files, columns=["path", "class", "resolution","data"])

data_dir = r'data/Split_smol/val/'
p = Path(data_dir).glob('**/*')
files = [(x, get_class(x), Image.open(x).size,Image.open(x)) for x in p if x.is_file()]
df_val = pd.DataFrame(files, columns=["path", "class", "resolution", "data"])

## Modelo para clasificación de imágenes con MLP

In [4]:
mlflow.set_experiment("CNN_dilation_search")

<Experiment: artifact_location='file:///home/herstegen/ITBA/redes/RN-TP1/mlruns/453648778997240401', creation_time=1751162886585, experiment_id='453648778997240401', last_update_time=1751162886585, lifecycle_stage='active', name='CNN_dilation_search', tags={}>

In [5]:
# Constant definitions

TRAIN_DIR = "data/Split_smol/train/"
VAL_DIR = "data/Split_smol/val/"

In [ ]:
hparams= {
    "model": ("MLPClassifier"),
    "input_size":  120,
    "batch_size": 32,
    "lr": 1e-3,
    "epochs": 200,
    "optimizer":  "Adam",
    "HFlip": 0.5,
    "VFlip": 0.5,
    "RBContrast": 0.5,
    "loss_fn": "CrossEntropyLoss",
    "train_dir": TRAIN_DIR,
    "val_dir": VAL_DIR,
    "es_patience": 10,
    "dropout": 0.1,
    "kernel_size": 2,
    "out_channels": 30,
    "dilation":3,
    "stride":2
}

In [7]:
log_dir = "runs/CNN_HP_ya_calculados"
writer = SummaryWriter(log_dir=log_dir)

# Train CNN

In [8]:
model_number = 0
# for input_size in hparams_space["input_size"]:
#     for kernel_size in hparams_space["kernel_size"]:
#         for out_channels in hparams_space["out_channels"]:
model_number += 1
# hparams= {
#     "model": ("MLPClassifier"),
#     "input_size":  input_size,
#     "batch_size": 32,
#     "lr": 1e-3,
#     "epochs": 200,
#     "optimizer":  "Adam",
#     "HFlip": 0.5,
#     "VFlip": 0.5,
#     "RBContrast": 0.5,
#     "loss_fn": "CrossEntropyLoss",
#     "train_dir": TRAIN_DIR,
#     "val_dir": VAL_DIR,
#     "es_patience": 10,
#     "dropout": 0.1,
#     "kernel_size": kernel_size,
#     "out_channels": out_channels,
#     "dilation":2,
#     "stride":1,
#     "model_number":model_number
# }

train_transform = A.Compose([
    A.Resize(hparams["input_size"], hparams["input_size"]),
    A.HorizontalFlip(p=hparams["HFlip"]),
    A.VerticalFlip(p=hparams["VFlip"]),
    A.RandomBrightnessContrast(p=hparams["RBContrast"]),
    A.Normalize(),
    ToTensorV2()
])
val_test_transform = A.Compose([
    A.Resize(hparams["input_size"], hparams["input_size"]),
    A.Normalize(),
    ToTensorV2()
])
train_dataset = CustomImageDataset(hparams["train_dir"], transform=train_transform)
val_dataset   = CustomImageDataset(hparams['val_dir'], transform=val_test_transform)
batch_size = hparams["batch_size"]
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(set(train_dataset.labels))
model = CNNClassifier(padding = 2, num_classes=num_classes, dilation=hparams["dilation"], input_size = hparams["input_size"], dropout = hparams["dropout"], kernel_size=hparams["kernel_size"],out_channels=hparams["out_channels"],stride=hparams["stride"]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.NAdam(model.parameters(), lr=hparams["lr"]) if hparams["optimizer"]=="Adam" else optim.SGD(model.parameters(), lr=hparams["lr"])
hparams["count_params"] = param_counter(model)
with mlflow.start_run():
    # Log hiperparámetros
    mlflow.log_params(hparams)
    best_val_acc = 0
    best_val_loss = 0
    best_train_acc = 0
    best_train_loss = 0
    best_epoch = 0
    for epoch in range(hparams["epochs"]):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0
    
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
    
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
        train_loss = running_loss / len(train_loader)
        train_acc = 100.0 * correct / total
        val_loss, val_acc = evaluate(model, val_loader, writer, device,train_dataset,criterion,epoch=epoch, prefix="val")
    
        print(f"Epoch {epoch+1}:")
        print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
        print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")
    
        writer.add_scalar("train/loss", train_loss, epoch)
        writer.add_scalar("train/accuracy", train_acc, epoch)
    
        # Log en MLflow
        mlflow.log_metrics({
            "train_loss": train_loss,
            "train_accuracy": train_acc,
            "val_loss": val_loss,
            "val_accuracy": val_acc
        }, step=epoch)

        del images, labels, outputs, preds      #intentamos no acumular memoria RAM

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_val_loss = val_loss
            best_train_acc = train_acc
            best_train_loss = train_loss
            best_epoch = epoch
            # Guardar modelo
            torch.save(model.state_dict(), "mlp_model.pth")
            #print("Modelo guardado como 'mlp_model.pth'")
            mlflow.log_artifact("mlp_model.pth")
            mlflow.pytorch.log_model(model, artifact_path="pytorch_model")
        elif epoch > best_epoch + hparams["es_patience"]:
            #print("Early Stopping")
            break
            
    mlflow.log_metrics({
            "train_loss": best_train_loss,
            "train_accuracy": best_train_acc,
            "val_loss": best_val_loss,
            "val_accuracy": best_val_acc,
            "best_epoch": best_epoch
        }, step=epoch+1)      
    



Epoch 1:
  Train Loss: 1.9808, Accuracy: 25.25%
  Val   Loss: 1.8304, Accuracy: 30.94%
Epoch 2:
  Train Loss: 1.5270, Accuracy: 43.04%
  Val   Loss: 1.4759, Accuracy: 39.23%
Epoch 3:
  Train Loss: 1.3347, Accuracy: 47.63%
  Val   Loss: 1.2509, Accuracy: 55.80%
Epoch 4:
  Train Loss: 1.1362, Accuracy: 55.67%
  Val   Loss: 1.1701, Accuracy: 54.14%
Epoch 5:
  Train Loss: 1.0446, Accuracy: 57.53%
  Val   Loss: 1.1671, Accuracy: 53.59%
Epoch 6:
  Train Loss: 0.9901, Accuracy: 60.11%
  Val   Loss: 1.1063, Accuracy: 54.70%
Epoch 7:
  Train Loss: 0.9056, Accuracy: 65.14%
  Val   Loss: 1.1031, Accuracy: 54.14%
Epoch 8:
  Train Loss: 0.8477, Accuracy: 65.71%
  Val   Loss: 1.1043, Accuracy: 59.67%
Epoch 9:
  Train Loss: 0.8860, Accuracy: 64.56%
  Val   Loss: 1.0395, Accuracy: 55.25%
Epoch 10:
  Train Loss: 0.7829, Accuracy: 69.58%
  Val   Loss: 1.0121, Accuracy: 56.91%
Epoch 11:
  Train Loss: 0.7705, Accuracy: 72.45%
  Val   Loss: 1.0716, Accuracy: 54.14%
Epoch 12:
  Train Loss: 0.7460, Accuracy: